# Higher-order functions

In this notebook, we will see that higher-order functions (HOFs) are essential modularity devices, and we will introduce the most common higher-order functions that operate over many different data structures.

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import _root_.org.scalatest._

import $ivy.$                               

import _root_.org.scalatest._

### References

[__Programming in Scala, 
A comprehensive step-by-step guide__](https://www.artima.com/shop/programming_in_scala_3ed) Third Edition.
by Martin Odersky, Lex Spoon, and Bill Venners. 

- Chapter 6. Functional objects
- Chapter 8. Functions and closures
- Chapter 16. Working with Lists
- Chapter 23. For Expressions Revisited

[__Functional programming in Scala__](https://www.manning.com/books/functional-programming-in-scala), by Paul Chiusano and Runar Bjarnason.

- Chapter 3. Functional data structures

[__Functional programming simplified__](https://alvinalexander.com/downloads/fpsimplified-free-preview.pdf), by Alvin Alexander.

- Chapter 22. Functions Are Variables, Too
- Chapter 23. Using Methods As If They Were Functions
- Chapter 24. How to Write Functions That Take Functions as Input Parameters
- Chapter 25. How to Write a ‘map’ Function
- Chapter 27. Functions Can Have Multiple Parameter Groups
- Chapter 28. Partially-Applied Functions (and Currying)

In [ ]:
def compose[A, B, C](f: B => C, g: A => B): A => C = 
    (a: A) => f(g(a))

In [ ]:
def curry[A, B, C](f: (A, B) => C): A => (B => C) = 
    ???

## `FoldRight`: divide and conquer

HOFs shine when the time comes to break monoliths. For instance, let's consider the following two functions:

In [ ]:
def sum(list: List[Int]): Int = 
    list match {
        case Nil => 0
        case (head: Int) :: (tail: List[Int]) => 
            val sumTail: Int = sum(tail)
            head + sumTail
    }

In [ ]:
def multiply(list: List[Int]): Int = 
    list match {
        case Nil => 1
        case (head: Int) :: (tail: List[Int]) => 
            val multiplyTail: Int = multiply(tail)
            head * multiplyTail
    }

These functions clearly share a common logic: their only differences are the value which is returned when the list is empty, and the function used to combine numbers (`+` and `*`, respectively). We can abstract away these differences and arrive to a more generic function which encodes that common logic:

In [ ]:
def foldRight(list: List[Int])(
    nil: Int, 
    cons: (Int, Int) => Int): Int = 
    list match {
        case Nil => nil
        case (head: Int) :: (tail: List[Int]) => 
            val tailSol: Int = foldRight(tail)(nil, cons)
            cons(head, tailSol): Int
    }

which allows us to re-define in a modular way the `sum` and `multiply` functions:

In [ ]:
def sum(list: List[Int]): Int = 
    foldRight(list)(
        0 , 
        (head: Int, tailSum: Int) => head + tailSum : Int
    )

In [ ]:
def sum(list: List[Int]): Int = 
    foldRight(list)(0 , _ + _)

In [ ]:
List(1,2,3,4,5).foldRight(0)(_ + _)

In [ ]:
sum(List(1,2,3,4,5))

In [ ]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        ??? : Int, 
        (head: Int, tailSol: Int) => 
            ??? : Int)

In [ ]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(
        1 : Int, 
        (head: Int, tailMultiplication: Int) => 
            head * tailMultiplication : Int)

In [ ]:
def multiply(list: List[Int]): Int = 
    foldRight(list)(1, _ * _)

But we don't need to constrain ourselves to integers. In its generic version, the `combine` function is actually the `foldRight` higher-order function (for `List`s):

In [ ]:
def sorted(list: List[Int]): Boolean = 
    list match {
        case Nil => true 
        case head :: tail => 
            val tailSorted: Boolean = sorted(tail)
            tail match {
                case Nil => true
                case head2 :: tail2 => 
                    head < head2 && tailSorted
            }
    }

In [ ]:
def sorted(list: List[Int]): Boolean = 
    list match {
        case head1 :: head2 :: tail => 
            head1 < head2 && sorted(head2 :: tail)
        case _ => true
    }

In [ ]:
sorted(List(1,6,2,3,4))

In [ ]:
def foldRight[A, B](list: List[A])(
    nil: B, 
    cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case (head: A) :: (tail: List[A]) => 
            val tailSol: B = foldRight(tail)(nil, cons)
            cons(head, tailSol): B
    }

In [ ]:
def sorted(list: List[Int]): Boolean = 
    foldRight(list)(
        ??? : Boolean, 
        (head: Int, tailSorted: Boolean) => 
            /*
            tail match {
                case Nil => true
                case head2 :: tail2 => 
                    head < head2 && tailSorted
            }
            */??? : Boolean)

In [ ]:
def sorted(list: List[Int]): Boolean = 
    foldRight(list)(
        (Nil, true) : (List[Int], Boolean), 
        (head: Int, tailSol: (List[Int], Boolean)) => 
            tailSol._1 match {
                case Nil => 
                    (head:: tailSol._1, true)
                case head2 :: tail2 => 
                    (head :: tailSol._1, 
                     head < head2 && tailSol._2)
            })._2

In [ ]:
sorted(List(1,2,7, 3,4,5)
      )

Graphically, we can explain the behaviour of `foldRight` as follows: 

![with elements](../images/foldRight.1.svg)

(all images credit to [Scala Visual Reference](https://superruzafa.github.io/visual-scala-reference/))

We can understand `foldRight` as an implementation of the divide-and-conquer design pattern: first, divide your problems in subproblems; second, solve these sub-problems; last, compose their solutions to obtain the solution to the overall problem. If the sub-problems are simple enough they can be solved directly; otherwise, they are solved recursively. In the case of lists:
- The problem is obtaining a value of type `B` for a given list.
- The only sub-problem corresponds to the tail of that list. 
- The arguments of the `foldRight` function tell us how to obtain the solution for the empty list (the atomic problem which can not be further decomposed), and how to obtain the solution from the solution to the sub-problem.

From this perspective, the implementation of `sum` and `multiply` is exactly the same as before when we used the function `combine`, although we can explain them differently:

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def multiply(list: List[Int]): Int = 
    ???

#### Improved type-inference

Technically, you may have noticed that we have to give extra type information in the invocations to `foldRight`. In fact, the following code doesn't compile. Check it yourself!

In [ ]:

def multiply(list: List[Int]): Int = 
    foldRight(list)(1)((a, b) => a * b)
    //list.foldRight(1)(_ * _)

In order to help the Scala compiler to infer the type parameters of the `foldRight` function, we need to change its signature a little bit: 

In [ ]:
def foldRight[A, B](list: List[A])(
    nil: B)( 
    cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case (head: A) :: (tail: List[A]) => 
            val tailSol: B = foldRight(tail)(nil)(cons)
            cons(head, tailSol): B
    }

By splitting the second parameter list, we allow the Scala compiler to infer the type of `B`, before it analyses the type of the `cons` argument. Now this works:

#### HOFs in the Scala API

The `foldRight` HOF, as all the HOFs that we will explain in this notebook, are actually part of the standard Collections library of Scala. Typically, they are implemented as member methods of the corresponding collection class. For lists, the [Scala API](https://www.scala-lang.org/api/2.13.1/scala/collection/immutable/List.html) offers something like this:

In [ ]:
object Standard{
    class List[A]{
        def foldRight[B](b: B)(f: (B, A) => B): B = ???
    }
}

So, invocations of the standard `foldRight` do not receive the list as argument, but are common method invocations on some `List` instance:

Normally, we will use standard HOFs instead of our home-made versions, once they are explained. 

## The Hall of Fame of HOFs

Besides `compose`, `andThen` and `foldRight`, there are other famous higher-order functions which work great as modularity devices: 

In [ ]:
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = ???
def foldLeft[A, B](list: List[A])(b: B)(f: (B, A) => B): B = ???
def filter[A](list: List[A])(f: A => Boolean): List[A] = ???
def map[A, B](list: List[A])(f: A => B): List[B] = ???
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = ???

But most of these functions do not only work for `List`, they work also for `Option`, `Either[A, ?]`, and many other data structures. For instance:

In [ ]:
def foldRight[A, B](opt: Option[A])(none: B)(some: A => B): B = ???
def filter[A](opt: Option[A])(f: A => Boolean): Option[A] = ???
def map[A, B](opt: Option[A])(f: A => B): Option[B] = ???  
def flatMap[A, B](opt: Option[A])(f: A => Option[B] ): Option[B] = ???

In [ ]:
def foldRight[A, B, C](opt: Either[A, B])(
    left: A => C, right: B => C): C = ???
def filter[A, B](opt: Either[A, B])(f: B => Boolean): Either[A, B] = ???
def map[A, B, C](opt: Either[A, B])(f: B => C): Either[A, C] = ???  
def flatMap[A, B, C](opt: Either[A, B])(f: B => Either[A, C] ): Either[A, C] = ???

The `fold` function is the catamorphism for `Option` and `Either[A, ?]`, in the same way than `foldRight` is the catamorphism for lists (we will see catamorphisms later on). 

## FoldLeft: a better loop

The `foldLeft` HOF is the functional way to implement common imperative algorithms. For instance:

In [ ]:
def length[A](list: List[A]): Int = {
    var out: Int = 0
    for (e <- list)
        out += 1
    out
}

In [ ]:
// out / aux
Nil
1 :: 2 :: 3 :: Nil
//
1 :: Nil
2 :: 3 :: Nil
// 
2 :: (1 :: Nil)
3 :: Nil
//
3 :: (2 :: (1 :: Nil))
Nil

In [ ]:
def reverse[A](list: List[A]): List[A] =  {
    var out: List[A] = Nil
    for (e <- list)
        out = e :: out
    out
}

The following implementation abstracts away the differences in the above functions, and declares two additional parameters: the initial value of a mutable variable, and the update function executed in every step of the loop.

In [ ]:
// def sorted(list: List[Int]): Boolean
// def length[A](list: List[A]): Int
// def reverse[A](list: List[A]): List[A]
def foldLeft[A, B](list: List[A])(
        initial: B,
        update: (B, A) => B): B =  {
    var out: B = initial
    for (e <- list)
        out = update(out: B, e)
    out
}

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    foldLeft(list)(
        Nil : List[A], 
        (e: A, out: List[A]) => 
            e :: out : List[A])

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    list.foldLeft[List[A]](
        Nil)(
        (out, e) => 
            e :: out)

In [ ]:
def length[A](list: List[A]): Int = 
    list.foldLeft(0)(
        (out: Int, _: A) => out + 1
    )

Similarly, the `foldLeft` function is typically used where a tail-recursive function is needed. Its recursive implementation is naturally tail-recursive:

In [ ]:
//@annotation.tailrec
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

Using `foldLeft` we can implement functions at a higher-level of abstraction, i.e. using constructs which are nearer to the problem-domain, without caring about mutable variables, tail-safe recursion, etc. Its use also leads to very concise (and readable!) implementations. For instance, these are (almost) one-liner implementations of some of the functions of the last notebook on recursive functions:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

But we may also use the `foldRight` function to implement them:

In [ ]:
def length[A](list: List[A]): Int = 
    ???

In [ ]:
def sum(list: List[Int]): Int = 
    ???

In [ ]:
def reverse[A](list: List[A]): List[A] = 
    ???

Functionally, both versions are equivalent, but the ones using `foldLeft` are more efficient. In particular, the `foldRight` version of `reverse` has cuadratic complexity. 

#### Implementing `foldRight` with `foldLeft`

The implementation that we gave for `foldRight` was not tail-recursive, so this will blow up the stack:

In [ ]:
foldRight[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

In order to obtain a better implementation, we may first reverse the list and use `foldLeft` as follows: 

In [ ]:
def foldRightTR[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    ???

This implementation works right with big lists:

In [ ]:
// foldRightTR[Int, Int](List.fill(100000)(1))(0)((_,_)=> 0)

#### Implementing `foldLeft` with `foldRight`

This is also possible, though challenging. We start from the tail-recursive implementation of `foldLeft`. First, we rearrange the arguments so that it now returns a function instead of a plain value of type `B`. 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    list match {
        case Nil => out
        case head :: tail => 
            foldLeft(tail)(update(out, head))(update)
    }

This was in order to better expose the recursion pattern of `foldRight`. The solution is now obtained simply by noticing that `foldLeft(tail)(update)` is the solution for the tail:

In [ ]:
def foldLeftAux[A, B](list: List[A])(update: (B, A) => B): B => B =
    ???

The original signature can then be implemented as follows: 

In [ ]:
def foldLeft[A, B](list: List[A])(out: B)(update: (B, A) => B): B =
    ???

## The `map` HOF

In [1]:
"llegáis tarde".length

res0: Int = 13

In [4]:
lengths(List("hola", "", "adios")) == List(4, 0, 5)

res3: Boolean = true

In [6]:
def evens(list: List[String]): List[Boolean] = 
    list match {
        case Nil => Nil : List[Boolean]
        case head :: tail => 
            val tailEvens: List[Boolean] = evens(tail)
            (head.length % 2 == 0) :: tailEvens : List[Boolean]
    }

defined function evens

In [8]:
def evens(list: List[String]): List[Boolean] = 
    map(list)((s : String) => (s.length % 2 == 0) : Boolean)

defined function evens

In [11]:
def evens(list: List[String]): List[Boolean] = 
    map(list)(_.length % 2 == 0)

defined function evens

In [13]:
def evens(list: List[String]): List[Boolean] = 
    list.map(_.length % 2 == 0)

defined function evens

In [3]:
def lengths(list: List[String]): List[Int] = 
    list match {
        case Nil => Nil : List[Int]
        case head :: tail => 
            val tailLengths: List[Int] = lengths(tail)
            head.length :: tailLengths : List[Int]
    }

defined function lengths

In [5]:
def lengths(list: List[String]): List[Int] = 
    list.foldRight(Nil : List[Int])( (head, tailLengths) => 
            head.length :: tailLengths : List[Int]
    )

defined function lengths

In [7]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    list match {
        case Nil => Nil : List[B]
        case (head: A) :: (tail: List[A]) => 
            val tailMapped: List[B] = map(tail)(f)
            f(head) :: tailMapped : List[B]
    }

defined function map

In [12]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    list.foldRight(Nil : List[B])( (head: A, tailMapped) => 
            f(head) :: tailMapped : List[B]
    )

defined function map

The `map` function is one of the essential HOFs. Its purpose is applying a function to the elements of a data structure, in such a way that the relationships between these elements do not change. The only thing that must be modified is the _content_ of the data structure, not its _shape_. This condition is expressed in the following equations:

1. `map(list)(identity) == list` for all `list: List[A]`
2. `map(map(list)(f))(g) == map(list)(g compose f)` for all `list: List[A]`, `f: A => B`, `g: B => C`

![map](../images/map.svg)

The implementation for lists can be done as follows:

In [ ]:
def map[A, B](list: List[A])(f: A => B): List[B] = 
    ???

The `map` function is polymorphic in `A` and `B`, but we can't pass generic functions (aka polymorphic values) as parameters using `FunctionN`classes (these classes only wrap functions from specific types to specific result types). We may create polymorphic versions of `FunctionN` clases, but in order to keep things simple, we will define the test catalogue for `map` using a specific signature chosen at random (any other may serve as well):

In [19]:
class TestMap(
    map: List[Int] => (Int => Boolean) => List[Boolean]
) extends FlatSpec with Matchers{
    
    val isEven: Int => Boolean = _ % 2 == 0
    val greaterThan0: Int => Boolean = _ > 0
    
    "mapping an empty list" should "return an empty list" in {
        map(List())(isEven) shouldBe Nil
    }
    
    "mapping an non-empty list" should "only change its content" in {
        map(List(1))(isEven) shouldBe List(false)
        map(List(1,2))(isEven) shouldBe 
            List(isEven(1),isEven(2))
        map(List(1,2,3))(_ % 2 == 0) shouldBe 
            List(isEven(1),isEven(2),isEven(3))
        map(List(1,2,3,-1))(_ > 0) shouldBe 
            List(greaterThan0(1),true,true,false)
    }
}

defined class TestMap

To run the tests we will pass the `map` function for `Ìnt` and `Boolean` types:

In [21]:
run(new TestMap(map))

cmd18$Helper$TestMap:
mapping an empty list
- should return an empty list
mapping an non-empty list
- should only change its content


The implementation of `map` is really close to the implementation of `foldRight`. Indeed, we can give a more modular implementation using this more basic HOF: 

In [ ]:
/*
def foldRight[A, B](list: List[A])(nil: B)(cons: (A, B) => B): B = 
    list match {
        case Nil => nil
        case head :: tail => cons(head, foldRight(tail)(nil)(cons))
    }
*/

def mapFR[A, B](list: List[A])(f: A => B): List[B] = 
    ???

In [ ]:
run(new TestMap(mapFR))

### `map` for `Option`

The implementation of the `map` function given above works over lists, but we can also map optional values and many other data structures. The implementation for the data structure `Option` must satisfy the following test:

In [22]:
class TestMapOption(
    map: Option[Int] => (Int => Boolean) => Option[Boolean]
) extends FlatSpec with Matchers{
    
    "mapping the `None` value" should "return `None`" in {
        map(None)(_ % 2 == 0) shouldBe None
    }
    
    "mapping a `Some` value" should "only change its content" in {
        map(Option(3))(_ % 2 == 0) shouldBe Option(false)
        map(Option(1))(_ % 2 == 0) shouldBe Option(false)
        map(Option(2))(_ % 2 == 0) shouldBe Option(true)
    }
    
        
}

defined class TestMapOption

The implementation is simple:

In [ ]:
def mapOpt[A, B](maybeA: Option[A])(f: A => B): Option[B] = 
    ???

In [ ]:
run(new TestMapOption(mapOpt))

## Filtering elements

Unlike `map`, the `filter` HOF allows us to change the shape of the data structure, removing those elements that do not satisfy a given predicate. 

![filter](../images/filter.svg)

In [25]:
def filterEven(list: List[Int]): List[Int] = 
    list.foldRight(Nil: List[Int])(
        (head: Int, tailFiltered: List[Int]) => 
            if (head % 2 == 0) head :: tailFiltered
            else tailFiltered : List[Int]
    )

defined function filterEven

In [32]:
filterEven(List(1,2,3,4))== List(2,4)

res31: Boolean = true

In [28]:
"hola"(0)
""(0)

: 

In [29]:
def startsWithVowel(s: String): Boolean = 
    s.length > 0 && (
        s(0) == 'a' || s(0)=='e' || s(0)=='i' || s(0)=='o' || s(0)=='u'
    )

defined function startsWithVowel

In [30]:
def filterVowel(list: List[String]): List[String] = 
    list.foldRight(Nil: List[String])(
        (head: String, tailFiltered: List[String]) => 
            if (startsWithVowel(head)) head :: tailFiltered
            else tailFiltered : List[String]
    )

defined function filterVowel

In [36]:
filterVowel(List("abc", "cdd", "baa", "eoo"))
filterR(List("abc", "cdd", "baa", "eoo"))(startsWithVowel)
List("abc", "cdd", "baa", "eoo").filter(startsWithVowel)
List("abc", "cdd", "baa", "eoo").map(startsWithVowel)

res35_0: List[String] = List("abc", "eoo")
res35_1: List[String] = List("abc", "eoo")
res35_2: List[String] = List("abc", "eoo")
res35_3: List[Boolean] = List(true, false, false, true)

In [39]:
def f: (String, List[Boolean]) => List[Boolean] = 
    (s: String, l: List[Boolean]) => ??? : List[Boolean]

defined function f

In [31]:
def filterR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    list.foldRight(Nil: List[A])(
        (head: A, tailFiltered: List[A]) => 
            if (predicate(head)) head :: tailFiltered
            else tailFiltered : List[A]
    )

defined function filterR

In [ ]:
class TestFilterList(
    filter: List[Int] => (Int => Boolean) => List[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(List())(_ % 2 == 0) shouldBe ???
        filter(List(1))(_ % 2 == 0) shouldBe ???
        filter(List(1,3,5))(_ % 2 == 0) shouldBe ???
        filter(List(2,4,6))(_ % 2 == 0) shouldBe ???
    }
}

In [ ]:
run(new TestFilterList(filter))

Using `foldRight` we can get a more modular implementation: 

In [ ]:
def filterFR[A](list: List[A])(predicate: A => Boolean): List[A] = 
    ???

In [ ]:
run(new TestFilterList(filterFR))

### Filtering optional values

We only have a possible value, so implementing filtering is easy in this case:

In [40]:
class TestFilterOption(
    filter: Option[Int] => (Int => Boolean) => Option[Int]
) extends FlatSpec with Matchers{
    
    "filter" should "work" in {
        filter(None)(_ % 2 == 0) shouldBe None
        filter(Some(1))(_ % 2 == 0) shouldBe None
        filter(Some(2))(_ % 2 == 0) shouldBe Some(2)
    }
}

defined class TestFilterOption

In [ ]:
def filter[A](maybeA: Option[A])(predicate: A => Boolean): Option[A] = 
    ???

In [ ]:
run(new TestFilterOption(filter))

## FlatMapping data structures

Let's consider a paragraph represented as a list of sentences, where each sentence is in turn modelled as a string made of _words_ separated by blank spaces. We want to obtain a list of all the words in each sentence. 

In [41]:
class TestWords(
    words: List[String] => List[String]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un lugar",
        "de la Mancha", 
        "de cuyo nombre no",
        "quiero acordarme")
    
    "words" should "work" in {
        words(paragraph1) shouldBe
            List(
        "En", "un", "lugar",
        "de", "la", "Mancha", 
        "de", "cuyo", "nombre", "no",
        "quiero", "acordarme")
            
    }
}

defined class TestWords

In [43]:
"en un lugar".split(" ").toList

res42: List[String] = List("en", "un", "lugar")

We may try to map the paragraph with a function that `split`s  each sentence into the words they are made of:

In [46]:
val paragraph: List[String] = List("en un lugar", "de la Mancha")

List("en", "un", "lugar", "de", "la", "Mancha") == 
 List("en", "un", "lugar") ++ List("de", "la", "Mancha")

paragraph.map[List[String]](
    (phrase: String) => 
        phrase.split(" ").toList: List[String]
)


paragraph: List[String] = List("en un lugar", "de la Mancha")
res45_1: Boolean = true
res45_2: List[List[String]] = List(
  List("en", "un", "lugar"),
  List("de", "la", "Mancha")
)

but then we obtain a _list of lists_ of strings, not a list of plain strings. The solution is not far though: we have just to concatenate all the lists and we obtain what we need. The function `flatten` performs this concatenation:

In [48]:
def concatenate(listOflists: List[List[String]]): List[String] = 
    listOflists.foldRight(
        Nil : List[String])(
        (head: List[String], tailConcatenated: List[String]) => 
            head ++ tailConcatenated : List[String]
    )

defined function concatenate

In [52]:
Some(Some(1)).flatten
Some(None).flatten

res51_0: Option[Int] = Some(1)
res51_1: Option[Nothing] = None

In [54]:
List(List(), List(), List()).flatten

res53: List[Nothing] = List()

In [55]:
def flatten[A](listOflists: List[List[A]]): List[A] = 
    listOflists.foldRight(
        Nil : List[A])(
        (head: List[A], tailConcatenated: List[A]) => 
            head ++ tailConcatenated : List[A]
    )

defined function flatten

In [49]:
def concatenate(listOflists: List[List[String]]): List[String] = 
    listOflists.foldRight(List[String]())(_ ++ _)

defined function concatenate

The function `flatten` is actually provided by the Scala API. So, the implementation of the `words` function is as follows:

In [56]:
def words(paragraph: List[String]): List[String] = 
    paragraph.map(_.split(" ").toList) // List[List[String]]
        .flatten                       // List[String]

defined function words

In [58]:
def words(paragraph: List[String]): List[String] = 
    paragraph.flatMap(_.split(" ").toList) // List[String]

defined function words

In [57]:
run(new TestWords(words))

cmd40$Helper$TestWords:
words
- should work


This combination of the HOF `map` and the function `flatten` is so common, that it has been given a proper name: `flatMap`. 

In [ ]:
def words(p: List[String]): List[String] = ???

def wordLengths(p: List[String]): List[Int] =
    p.map(???: String => List[Int]) // List[List[Int]]
        .flatten                  // List[Int]

def wordLengths(p: List[String]): List[Int] =
    p.flatMap(??? : String => List[Int]) // List[Int]

In [60]:
def flatMap[A, B](list: List[A])(f: A => List[B]): List[B] = 
    list.map(f: A => List[B]) // List[List[A]]
        .flatten              // List[A]

defined function flatMap

![filter](../images/flatMap.svg)

Using `flatMap` the word function is implemented even more easily:

In [ ]:
def words(paragraph: List[String]): List[String] = 
    ???

### FlatMapping optional values

In [ ]:
def flatMap[A, B](maybeA: Option[A])(cont: A => Option[B]): Option[B] = 
    ???

In [61]:
class TestFlatMapOption(
    flatMap: Option[Int] => (Int => Option[Boolean]) => Option[Boolean]
) extends FlatSpec with Matchers {
    
    def isPositiveEven(i: Int): Option[Boolean] = 
        if (i>=0) Some(i % 2 == 0)
        else None
    
    "flatMap" should "work" in {
        flatMap(None)(isPositiveEven) shouldBe ???
        flatMap(Some(5))(isPositiveEven) shouldBe Some(false)
        flatMap(Some(-5))(isPositiveEven) shouldBe None
        flatMap(Some(0))(isPositiveEven) shouldBe Some(true)
        flatMap(Some(4))(isPositiveEven) shouldBe Some(true )     
    }
}

defined class TestFlatMapOption

In [ ]:
run(new TestFlatMapOption(flatMap))

## All together: a modularity problem

As before, we start from a list of sentences, but now we want to take care of the possible extra spaces between each word. Also, we want to obtain not the words themselves but their lengths. So, the specification of our problem is a function `lengths`:

In [5]:
class TestLengths(
    lengths: List[String] => List[Int]
) extends FlatSpec with Matchers{
            
    val paragraph1 = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")
    
    "lengths" should "work" in {
        lengths(paragraph1) shouldBe 
            List(
        2, 2,  5,
        2,  2, 6, 
        2, 4, 6, 2,
        6, 9)
    }
}

defined class TestLengths

We may try to implement this function imperatively using mutable variables:

In [9]:
"en  un lugar". split(" ").toList

res8: List[String] = List("en", "", "un", "lugar")

In [12]:
def lengthsM(paragraph: List[String]): List[Int] = {
    var out: List[Int] = List()
    for (sentence <- paragraph)
        for (word <- sentence.split(" "))
            if (word != "") 
                out = out :+ word.length
    //out.reverse
    out
}


defined function lengthsM

and this works, of course: 

In [13]:
run(new TestLengths(lengthsM))

cmd4$Helper$TestLengths:
lengths
- should work


but can we do it better? Yes, we can! Using HOFs we can get a more concise, understandable and reliable version, with a great level of reuse!

In [16]:
List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")                   // List[Sentence]
    .flatMap(sentence => sentence.split(" ").toList) // List[Word]
    .filter(word => word != "" : Boolean)            // List[Word]
    .map(word => word.length)                        // List[Int]
    

res15: List[Int] = List(2, 2, 5, 2, 2, 6, 2, 4, 6, 2, 6, 9)

In [ ]:
def lengthsHOF(paragraph: List[String]): List[Int] = 
    paragraph.flatMap()

In [ ]:
run(new TestLengths(lengthsHOF))

This solution contrasts with the mutable version in several respects:
- It's more **modular**, i.e. it's made of coarse-grained components: the HOFs `flatMap`, `filter` and `map`. The mutable version builds instead upon fine-grained components: `var`s, and `for` loops.
- The HOF components `flatMap`, etc., are actually very generic and domain-independent, and are typically part of standard libraries, so the level of **reuse** and **reliability** of the HOF-based solution is very high.
- The HOF-based solution is more **understandable**: it models the solution to the problem in terms of standard _transformations_ which are composed together using plain function composition. Moreover, these transformations are also at the right level of abstraction, i.e. it's natural to specify the solution to the problem in terms of flatMap, filter and map.

### For-comprehensions

The combination of `map`, `flatMap` and `filter` HOFs is so common, that Scala has a special syntax for them: for-comprehensions.

In [18]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph.flatMap(sentence => sentence.split(" ").toList) // List[Word]
        .filter(word => word != "" : Boolean)            // List[Word]
        .map(word => word.length)                        // List[Int]


defined function lengthsFC

In [20]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph.flatMap(_.split(" ").toList) // List[Word]
        .filter(_ != "")                   // List[Word]
        .map(_.length)                     // List[Int]


defined function lengthsFC

In [32]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph                              // List[Sentence]
        .flatMap(_.split(" ").toList)      // List[Word]
        .map(_.length)                     // List[Int]
        .filter(_ != 0)                   // List[Int]
        


defined function lengthsFC

In [35]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph                              // List[Sentence]
        .flatMap(sentence => 
            sentence.split(" ").toList
               .map(word => word.length)
               .filter(longitud => longitud != 0)// List[Int]
        ) // List[Int]
        


defined function lengthsFC

In [28]:
val paragraph = List(
        "En un  lugar",
        "de  la Mancha ", 
        "de cuyo nombre no",
        "quiero        acordarme")

paragraph: List[String] = List(
  "En un  lugar",
  "de  la Mancha ",
  "de cuyo nombre no",
  "quiero        acordarme"
)

In [30]:
paragraph
    .flatMap(s => s.split(" ").toList)
    .filter(w => w != "")    
    .map(w => w.length)


res29: List[Int] = List(2, 2, 5, 2, 2, 6, 2, 4, 6, 2, 6, 9)

In [31]:
for {
    sentence <- paragraph 
    word <- sentence.split(" ").toList if word != "" 
} yield word.length

res30: List[Int] = List(2, 2, 5, 2, 2, 6, 2, 4, 6, 2, 6, 9)

In [20]:
def lengthsFC(paragraph: List[String]): List[Int] = 
    paragraph.flatMap(_.split(" ").toList) // List[Word]
        .filter(_ != "")                   // List[Word]
        .map(_.length)                     // List[Int]


defined function lengthsFC

In [34]:
run(new TestLengths(lengthsFC))

cmd4$Helper$TestLengths:
lengths
- should work
